## Updating Constraint Matrices

To implement Graph SLAM, a matrix and a vector (omega and xi, respectively) are introduced. The matrix is square and labelled with all the robot poses (xi) and all the landmarks (Li). Every time you make an observation, for example, as you move between two poses by some distance `dx` and can relate those two positions, you can represent this as a numerical relationship in these matrices.

Below you can see a matrix representation of omega and a vector representation of xi.

<img src='images/omega_xi.png' width=20% height=20% />


### Solving for x, L

To "solve" for all these poses and landmark positions, we can use linear algebra; all the positional values are in the vector `mu` which can be calculated as a product of the inverse of omega times xi.

---


## Constraint Updates

In the below code, we construct `omega` and `xi` constraint matrices, and update these according to landmark sensor measurements and motion.

#### Sensor Measurements

When you sense a distance, `dl`, between a pose and a landmark, l, update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `l`
* Add `-dl` and `dl` to xi at the rows for `xt` and `l`

The values 2 instead of 1 indicate the "strength" of the measurement.

You'll see three new `dl`'s as new inputs to our function `Z0, Z1, Z2`, below.

#### Motion
When your robot moves by some amount `dx` update the constraint matrices as follows:
* Add `[[1, -1], [-1, 1]]` to omega at the indices for the intersection of `xt` and `xt+1`
* Add `-dx` and `dx` to xi at the rows for `xt` and `xt+1`

## QUIZ: Include three new sensor measurements for a single landmark, L.

In [ ]:
import numpy as np


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):
    
    ## TODO: construct constraint matrices
    ## and add each position/motion constraint to them
    
    # initialize constraint matrices with 0's
    # Now these are 4x4 because of 3 poses and a landmark
    omega = np.zeros((4,4))
    xi = np.zeros((4,1))
    
    # add initial pose constraint
    omega[0][0] = 1
    xi[0] = initial_pos
    
    # account for the first motion, dx = move1
    omega += [[1., -1., 0., 0.],
              [-1., 1., 0., 0.],
              [0., 0., 0., 0.],
              [0., 0., 0., 0.]]
    xi += [[-move1],
           [move1],
           [0.],
           [0.]]
    
    # account for the second motion
    omega += [[0., 0., 0., 0.],
              [0., 1., -1., 0.],
              [0., -1., 1., 0.],
              [0., 0., 0., 0.]]
    xi += [[0.],
           [-move2],
           [move2],
           [0.]]
    
    
    ## TODO: Include three new sensor measurements for the landmark, L
    
    # Your code here
    
    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')
    
    ## TODO: calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    omega_inv = np.linalg.inv(np.matrix(omega))
    mu = omega_inv*xi
    return mu

In [5]:
import numpy as np


def mu_from_positions(initial_pos, move1, move2, Z0, Z1, Z2):
    
    ## construct constraint matrices
    ## and add each position/motion constraint to them
    
    # initialize omega, xi
    n = 4 # [x0, x1, x2, l0]
    omega = np.zeros((n,n))
    xi = np.zeros((n,1))

    def add_initial_condition(omega, xi, initial_pos):
        # x0 = initial_pos
        omega[0][0] += 1
        xi[0] += initial_pos
        return omega, xi

    def add_motion_constraint(omega, xi, k, motion):
        # motion
        # x_k-x_(k-1) = motion
        omega[k][k+1] += 1
        omega[k][k] -= 1
        xi[k] += motion
        # -x_k+x_(k-1) = -motion
        omega[k+1][k] += 1
        omega[k+1][k+1] -= 1
        xi[k+1] -= motion
        return omega, xi

    def add_landmark_constraint(omega, xi, k, m, position):
        # landmark observation
        # l_k-x_k = position
        omega[k][k] -= 1
        omega[k][m] += 1
        xi[k] += position
        # -l_k+x_k = -position
        omega[m][k] += 1
        omega[m][m] -= 1
        xi[m] -= position
        return omega, xi

    omega, xi = add_initial_condition(omega, xi, initial_pos)
    omega, xi = add_landmark_constraint(omega, xi, 0, 3, Z0)
    
    omega, xi = add_motion_constraint(omega, xi, 0, move1)
    omega, xi = add_landmark_constraint(omega, xi, 1, 3, Z1)

    omega, xi = add_motion_constraint(omega, xi, 1, move2)
    omega, xi = add_landmark_constraint(omega, xi, 2, 3, Z2)


    # display final omega and xi
    print('Omega: \n', omega)
    print('\n')
    print('Xi: \n', xi)
    print('\n')
    
    ## calculate mu as the inverse of omega * xi
    ## recommended that you use: np.linalg.inv(np.matrix(omega)) to calculate the inverse
    #mu = np.linalg.inv(omega) * xi.transpose()
    mu = np.linalg.inv(omega) @ xi
    #mu = np.matmul(np.linalg.inv(omega), xi)
    return mu


In [6]:
# call function and print out `mu`
mu = mu_from_positions(-3, 5, 3, 10, 5, 2)
print('Mu: \n', mu)

Omega: 
 [[-1.  1.  0.  1.]
 [ 1. -3.  1.  1.]
 [ 0.  1. -2.  1.]
 [ 1.  1.  1. -3.]]


Xi: 
 [[ 12.]
 [  3.]
 [ -1.]
 [-17.]]


Mu: 
 [[-3.]
 [ 2.]
 [ 5.]
 [ 7.]]


In [8]:
# call function and print out `mu`
mu = mu_from_positions(-3, 2, 5, 10, 5, 1)
print('Mu: \n', mu)

Omega: 
 [[-1.  1.  0.  1.]
 [ 1. -3.  1.  1.]
 [ 0.  1. -2.  1.]
 [ 1.  1.  1. -3.]]


Xi: 
 [[  9.]
 [  8.]
 [ -4.]
 [-16.]]


Mu: 
 [[-3.]
 [ 0.]
 [ 5.]
 [ 6.]]
